In [1]:
import cv2
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import numpy as np
import matplotlib.pyplot as plt
from numpy import loadtxt
import warnings
warnings.filterwarnings("ignore")
from keras.models import load_model

### Resnet50V2 (128 x 128)

In [2]:
trained_model_path = '../notebooks/models/resnet_fine_tunned_lrdecay.h5'

In [3]:
model = load_model(trained_model_path)

In [4]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 4, 4, 2048)        23564800  
                                                                 
 flatten_1 (Flatten)         (None, 32768)             0         
                                                                 
 batch_normalization_2 (Batc  (None, 32768)            131072    
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 128)               4194432   
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 128)              512       
 hNormalization)                                      

In [5]:
def predictions(model, frame, labels_dict, dimension, channels):
    data = np.ndarray(shape=(1, dimension, dimension, channels))
    data[0] = frame
    prediction = model.predict(data, verbose=0)
    index = np.argmax(prediction)
    confidence_score = prediction[0][index]
    label = labels_dict[index]
    confidence_score_str = str(float("{:.2f}".format(confidence_score*100)))+"%"
    return label, confidence_score_str

In [6]:
def reshape_dimension(frame, dimension, channels):
    image = cv2.resize(frame,(dimension, dimension))
    resized = image.reshape(dimension, dimension, channels) / 255
    return resized

### Getting images

In [7]:
labels = {}
letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'nothing', 'space']
n = 0
for label in letters:
    labels[n] = label
    n += 1

In [8]:
vid = cv2.VideoCapture(0)

while(True):
    upper_left = (50, 80)
    bottom_right = (270, 300)
    
    ret, image_frame = vid.read()
    r = cv2.rectangle(image_frame, upper_left, bottom_right, (10, 230, 10), 4)
    rect_img = image_frame[upper_left[1] : bottom_right[1], upper_left[0] : bottom_right[0]]
    
    image_frame[upper_left[1] : bottom_right[1], upper_left[0] : bottom_right[0]] = rect_img
    reshaped_image = reshape_dimension(rect_img, 128, 3)
    
    a = reshaped_image
    
    label, confidence_score = predictions(model, reshaped_image, labels, 128, 3)
    
    cv2.putText(image_frame, label, (50,72), cv2.FONT_HERSHEY_DUPLEX , 1.1, (0,255,0), 2)
    cv2.putText(image_frame, confidence_score, (205,72), cv2.FONT_HERSHEY_DUPLEX, 0.6, (0,255,0), 2)
    cv2.imshow('frame', image_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break